In [1]:
# Cuts used to go from o.summary.fits to o.gst.fits, for now we will use the o.gst.fits file

#snr = 5.0
#sharp = 0.04
#crowd = 0.5
#objtype = 1
#flag = 99

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
import numpy as np
import os


In [2]:
def writeregionfile(filename, objlist, color="blue",sys=''):
    if sys == '': sys = 'wcs'
    out = open(filename,'w')
    i = -1
    out.write('# Region file format: DS9 version 4.0\nglobal color='+color+' font="helvetica 10 normal" select=1 highlite=1 edit=1 move=1 delete=1 include=1 fixed=0 source\n')
    if sys == 'wcs':
      out.write('fk5\n')
      for i in range(len(objlist)):
        out.write("point(%.7f,%.7f) # point=boxcircle text={%i}\n" % (objlist.RA[i],objlist.DEC[i], i))
    if sys == 'img':
      out.write('image\n')
      for i in range(len(objlist)):
        out.write("point(%.3f,%.3f) # point=boxcircle text={%i}\n" % (objlist.X[i],objlist.Y[i], i))
    out.close()
    print("writing region file:", filename)


In [12]:
def find_attributes(filename, radius):
    
    #find name
    global sn_name
    if '2021' in filename:
        sn_name = '2021'
        for i in range(len(filename)):
            if filename[i+4] == '-':
                print('SN name: ' + sn_name)
                break
            sn_name += filename[i+4]
            #print('adding char')
    else:
        print('year not found')


    #find filt
    global filt
    idx_start = len(sn_name) + 1
    if 'F' in filename:
        filt = ''
        for i in range(len(filename)):
            if filename[idx_start + i] == '-':
                print('Filt: ' + filt)
                break   
            filt += filename[idx_start + i]
            #print('adding char')
    else:
        print('filt not found')
    
    #find mjd    
    global mjd
    idx_start = len(sn_name) + len(filt) + 2
    if 'mjd' in filename:
        global mjd
        mjd = ''
        for i in range(len(filename)):
            if filename[idx_start + i + 4] == '-':
                print("MJD: " + mjd)
                break  
            mjd += filename[idx_start + i + 4]
            #print('adding char')
    else:
        print('mjd not found')
    global search_radius
    search_radius = radius * u.arcsec
    print('Search radius: ' + str(search_radius))

In [13]:
#SN2021sjt
sn_position = SkyCoord(309.33000691667*u.degree, +66.106417016667*u.degree)
path = '/Users/Dominic/Lamat/PROGRAMS/test_photometry/photometry_catalogs/'
sn_catalog = '2021sjt-F547M-mjd_49550-cut.fits'

find_attributes('2021sjt-F547M-mjd_49550-cut.fits', 3)


#sn_name='2021sjt'
#filt='F555W'
#mjd='56789'
#search_radius = 3 * u.arcsec

print(sn_name)
print(filt)
print(mjd)

SN name: 2021sjt
Filt: F547M
MJD: 49550
Search radius: 3.0 arcsec
2021sjt
F547M
49550


In [14]:
# Read in the photometry for the filtered catalog and the summary catalog
cut_catalog=fits.open(path+sn_catalog)[1]
full_catalog=fits.open(path+sn_catalog)[1]

data_cut=cut_catalog.data
data_full=full_catalog.data

tbl_cut=Table(data_cut)
tbl_full=Table(data_full)

radec_cut = SkyCoord(ra=data_cut.RA*u.degree, dec=data_cut.DEC*u.degree)
radec_full = SkyCoord(ra=data_full.RA*u.degree, dec=data_full.DEC*u.degree)

In [15]:
d2d_cut = sn_position.separation(radec_cut)
d2d_full = sn_position.separation(radec_full)

index_cut=d2d_cut < search_radius
index_full=d2d_full < search_radius

data_searched_cut=data_cut[index_cut]
data_searched_full=data_full[index_full]

tbl_searched_cut=tbl_cut[index_cut]
tbl_searched_full=tbl_full[index_full]


In [16]:
writeregionfile(sn_name+"-"+filt+"-mjd_"+mjd+"-"+str(search_radius.value) +"_arcsec-ra_dec.reg", data_searched_cut, "red", "wcs")
writeregionfile(sn_name+"-"+filt+"-mjd_"+mjd+"-"+str(search_radius.value) +"_arcsec-pixel_xy.reg", data_searched_cut, "red", "img")

writing region file: 2021sjt-F547M-mjd_49550-3.0_arcsec-ra_dec.reg
writing region file: 2021sjt-F547M-mjd_49550-3.0_arcsec-pixel_xy.reg


In [16]:
global sn_name
global search_radius
global filt

find_attributes('2021gmj-F814W-mjd_59574-summary.fits', 3)
find_attributes('2021sjt-F555W-mjd_57352-summary.fits', 3)

SN name: 2021gmj
Filt: F814W
MJD: 59574
Search radius: 3.0 arcsec
SN name: 2021sjt
Filt: F555W
MJD: 57352
Search radius: 3.0 arcsec


In [17]:
def returnregionfiles(path, rai, deci, radius):
    dir_list = os.listdir(path)
    #for each file in path
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        print(filename)
        find_attributes(filename, radius)
        sn_position = SkyCoord(rai*u.degree, +deci*u.degree)
        cut_catalog=fits.open(path+sn_catalog)[1]
        data_cut=cut_catalog.data
        tbl_cut=Table(data_cut)
        radec_cut = SkyCoord(ra=data_cut.RA*u.degree, dec=data_cut.DEC*u.degree)
        d2d_cut = sn_position.separation(radec_cut)
        index_cut= d2d_cut < search_radius
        data_searched_cut=data_cut[index_cut]
        tbl_searched_cut=tbl_cut[index_cut]
        writeregionfile(sn_name+"-"+filt+"-mjd_"+mjd+"-"+str(search_radius.value) +"_arcsec-ra_dec.reg", data_searched_cut, "red", "wcs")
        writeregionfile(sn_name+"-"+filt+"-mjd_"+mjd+"-"+str(search_radius.value) +"_arcsec-pixel_xy.reg", data_searched_cut, "red", "img")

In [18]:
returnregionfiles(path, 309.33000691667, 66.106417016667, 3)

2021sjt-F110W-mjd_57452-cut.fits
SN name: 2021sjt
Filt: F110W
MJD: 57452
Search radius: 3.0 arcsec
writing region file: 2021sjt-F110W-mjd_57452-3.0_arcsec-ra_dec.reg
writing region file: 2021sjt-F110W-mjd_57452-3.0_arcsec-pixel_xy.reg
2021sjt-F547M-mjd_49550-summary.fits
SN name: 2021sjt
Filt: F547M
MJD: 49550
Search radius: 3.0 arcsec
writing region file: 2021sjt-F547M-mjd_49550-3.0_arcsec-ra_dec.reg
writing region file: 2021sjt-F547M-mjd_49550-3.0_arcsec-pixel_xy.reg
2021sjt-F555W-mjd_59627-summary.fits
SN name: 2021sjt
Filt: F555W
MJD: 59627
Search radius: 3.0 arcsec
writing region file: 2021sjt-F555W-mjd_59627-3.0_arcsec-ra_dec.reg
writing region file: 2021sjt-F555W-mjd_59627-3.0_arcsec-pixel_xy.reg
2021gmj-F625W-mjd_59574-summary.fits
SN name: 2021gmj
Filt: F625W
MJD: 59574
Search radius: 3.0 arcsec
writing region file: 2021gmj-F625W-mjd_59574-3.0_arcsec-ra_dec.reg
writing region file: 2021gmj-F625W-mjd_59574-3.0_arcsec-pixel_xy.reg
2021sjt-F555W-mjd_57676-cut.fits
SN name: 2021sj